In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
from PIL import Image

In [15]:
class flowerModel():
    def __init__(self):
        self.model = models.resnet18(pretrained=True)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def transform_image(self):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])
        ])
        return transform

    def load_data(self, path):
        dataset = datasets.ImageFolder(root=path, transform=self.transform_image())
        self.classes = dataset.classes

        train_size = int(len(dataset) * 0.8)
        validation_size = len(dataset) - train_size
        train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

        # Replace the last layer now that we know num_classes
        num_classes = len(self.classes)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.model.to(self.device)

        self.train(train_loader, validation_loader, epochs=10)

    def train(self, train_loader, validation_loader, epochs=10):
        for epoch in range(epochs):
            self.model.train()
            running_loss = 0.0
            for images, labels in train_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

    def predict(self, image_path):
        image = Image.open(image_path).convert("RGB")
        img_t = self.transform_image()(image)
        img_t = img_t.unsqueeze(0).to(self.device)

        self.model.eval()
        with torch.no_grad():
            outputs = self.model(img_t)
            _, predicted = torch.max(outputs, 1)

        class_name = self.classes[predicted.item()]
        print("Predicted class:", class_name)


In [ ]:
flower = flowerModel()
# don't change this path
flower.load_data("./flowers/") # train data path


# add image path to predict
flower.predict("./rose.jpg")    # test image path

/home/nitin/coding/Projects/plant_detection/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nitin/coding/Projects/plant_detection/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.6756
Epoch 2, Loss: 0.4544
